In [1]:
debug = False

In [2]:
# global packages
import gymnasium as gym
import random
import numpy as np

# ML packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

#SB3
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common import logger
from stable_baselines3.common.logger import configure

# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Video
import glob
import io
import base64
import imageio
import gymnasium as gym
from IPython.display import HTML, display
import os

# Analysis
import pandas as pd
from itertools import product

In [3]:
# Base path for local storage
base_path = "/Users/maartendoekhie/Desktop/lunar_lander"

# Folder structure
folders = {
    "models": os.path.join(base_path, "models"),
    "videos": os.path.join(base_path, "videos"),
    "results": os.path.join(base_path, "results"),
    "logs": os.path.join(base_path, "logs"),
    "plots": os.path.join(base_path, "plots"),
    "logs_dqn": os.path.join(base_path, "logs", "dqn_tensorboard"),
    "logs_a2c": os.path.join(base_path, "logs", "a2c_tensorboard"),
}

# Create directories if they don't exist
for path in folders.values():
    os.makedirs(path, exist_ok=True)

In [4]:
# First create your custom environment
env = gym.make(
    "LunarLander-v3",
    render_mode="rgb_array",
    continuous=False,
    gravity=-10.0,
    enable_wind=False,
    wind_power=15.0,
    turbulence_power=1.5,
)

# Wrap in DummyVecEnv and then VecMonitor
env_vec = DummyVecEnv([lambda: env])
env_vec = VecMonitor(env_vec)

In [5]:
# Action space: What can your agent do?

# 0: do nothing
# 1: fire left orientation engine
# 2: fire main engine
# 3: fire right orientation engine

action = env.action_space # not needed for now
action_size = 4

# Observation space: What can your agent see?

# The state is an 8-dimensional vector:

# the coordinates of the lander in x & y,
# its linear velocities in x & y, its angle,
# its angular velocity,
# and two booleans that represent whether each leg is in contact with the ground or not.

state = env.observation_space.shape # not needed for now
state_size = 8

### Vectorized version of the environment ### 

In [6]:
### HYPER PARAMETERS ###

if debug == True:
  # Replay Memory
  replay_buffer_size = 10000 
  minibatch = 32

  # learning
  learning_rate = 5e-4
  gamma = 0.99 
  interpolation_parameter = 1e-3

  # training
  number_episodes = 5 
  max_time_steps = 200

  # epsilon gready policy
  epsilon_starting_value = 1.0
  epsilon_ending_value = 0.01
  epsilon_decay_value = 0.995

  # evaluation
  n_eval_episodes = 3

  # SB3
  total_timesteps = 10000 
  batch_size = minibatch
  buffer_size = replay_buffer_size

else: 
  # Replay Memory
  replay_buffer_size = 100000
  minibatch = 150

  # learning
  learning_rate = 5e-4
  gamma = 0.99 
  interpolation_parameter = 1e-3 

  # training
  number_episodes = 5000
  max_time_steps = 1000

  # epsilon gready policy
  epsilon_starting_value = 1.0
  epsilon_ending_value = 0.01
  epsilon_decay_value = 0.995

  # evaluation
  n_eval_episodes = 10

  # SB3
  total_timesteps = 10000000
  batch_size = minibatch
  buffer_size = replay_buffer_size

In [7]:
##############################
### CUSTOM DQN AGENT SETUP ###
##############################

# Consisting of the DQN model, Agent (with implemented learning and training), Replay Memory

### DQN MODEL SETUP ###

# We wan't to start with a simple 2 layer fc NN. Starting from state size to action size
# Shallow problems 1 to 2 hidden layers and complex tasks 3 to 4 hidden layers
# Underfitting: add more layers/neurons, overfitting: add droppout, regularization or reduce size
# Slow learning: reduce depth or learning rate

class DQN_custom(nn.Module):
  def __init__(self, state_size, action_size, seed = 4):
    super(DQN_custom, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.model = nn.Sequential(
        nn.Linear(state_size, 64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64, action_size)
    )
  def forward(self, state):
    return self.model(state)

# the memory can store an event: s, a,r, s' and done (with a maximum capacity: i.e. hyper parameter)
# the memory can give a sample: s, a, r, s' and done in a minibatch. i.e. matrices: S, A, R, R_, D

# 1. s:      the current state (before the action was taken)
#            This is the input to the Q-network and represents the agent’s current observation of the environment.

# 2. a:      the action taken by the agent in state s
#            This action is chosen according to an exploration policy (e.g., ε-greedy) and used to interact with the environment.

# 3. r:      the reward received after taking action a in state s
#            This tells the agent how good or bad the outcome of that action was.

# 4. s':     the next state observed after taking action a
#            This is the new observation received from the environment after applying the action.

# 5. done:   a boolean flag indicating if the episode ended after this step
#            If done is True, then s' is the terminal state and no further actions should be taken.


### DQN MEMORY ###

class Memory(object):

  def __init__(self, capacity):
    self.capacity = capacity
    self.memory = deque(maxlen=capacity)

  def store(self, event):
    self.memory.append(event)

  def sample(self, batch_size):
    if len(self.memory) < batch_size:# Not enough samples yet: tell the agent to skip learning

      return None

    else:
      minibatch = random.sample(self.memory, batch_size)
      state_list      = []
      action_list     = []
      reward_list     = []
      next_state_list = []
      done_list       = []

      for experiences in minibatch: # for every experience in the minibatch, we have a sars', done.
        if experiences is not None:
          state, action, reward, next_state, done = experiences
          state_list.append(state)
          action_list.append(action)
          reward_list.append(reward)
          next_state_list.append(next_state)
          done_list.append(done)

          # 1. states:      shape = (batch_size, 8)
          #    Each row is a full state vector from LunarLander (8 floats: x, y, vx, vy, angle, angular_vel, leg1_contact, leg2_contact)

          # 2. actions:     shape = (batch_size, 1)
          #    Each row contains the action taken (integer 0–3), one per experience

          # 3. rewards:     shape = (batch_size, 1)
          #    Each row contains the scalar reward received after taking the action

          # 4. next_states: shape = (batch_size, 8)
          #    Each row is the resulting state after the action was taken (same format as states)

          # 5. dones:       shape = (batch_size, 1)
          #    Each row is 1.0 if the episode ended after this transition, else 0.0

          # We wan't to create matrices of these events. with size explained above

      S = torch.from_numpy(np.vstack(state_list)).float()
      A = torch.from_numpy(np.vstack(action_list)).long()
      R = torch.from_numpy(np.vstack(reward_list)).float()
      R_ = torch.from_numpy(np.vstack(next_state_list)).float()
      D = torch.from_numpy(np.vstack(done_list).astype(np.uint8)).float()

      return S, A, R, R_, D

### DQN AGENT ###

class Agent():

  def __init__(self, state_size, action_size):
      self.action_size = action_size
      self.state_size = state_size

      self.local_qnetwork = DQN_custom(state_size, action_size)
      self.target_qnetwork = DQN_custom(state_size, action_size)

      self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
      self.memory = Memory(replay_buffer_size)
      self.t_step = 0

  def step(self, state, action, reward, next_state, done):
      self.memory.store((state, action, reward, next_state, done))
      self.t_step = (self.t_step + 1) % 4
      if self.t_step == 0:
          experiences = self.memory.sample(minibatch)
          if experiences is not None:
              self.learn(experiences, gamma)

  def get_action(self, state, epsilon):
    state = torch.from_numpy(state).float().unsqueeze(0) # pytorch expects a tensor batch
    self.local_qnetwork.eval() # avoid unwanted updates

    with torch.no_grad(): # passing the state through the network without calculating gradients.
        action_values = self.local_qnetwork(state)  # [Q(state, action_0), Q(state, action_1), ...]
    self.local_qnetwork.train() # putting network back into the training network

    if random.random() > epsilon: # exploration, exploitation tradeoff, epsilon-greedy exploration?
        return np.argmax(action_values.cpu().data.numpy()) # get the action with the highest q-value, exploitation part
    else:
        return random.choice(np.arange(self.action_size)) # else the agent will pick a random move. This happens when the agent, decides to explore rather than exploit.

  def learn(self, experiences, gamma): # training an AI agent in reinforcement learning, bellman adaptation for deep q learning is used (maybe here use something from the lecture slides)

      states, actions, rewards, next_states, dones = experiences

      next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1) # Check pictures

      q_targets = self.bellman_optimality(rewards, next_q_targets, dones, gamma) # Bellman equation on the target network,

      q_expected = self.local_qnetwork(states).gather(1, actions) # expected calculaton on the local network

      loss = nn.MSELoss() # (Q_target - Q_expected)^2
      loss = loss(q_expected, q_targets)

      self.optimizer.zero_grad() # classic!
      loss.backward()
      self.optimizer.step()

      self.td_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter) # See temporal difference updating from slides > implement in report.

  def bellman_optimality(self, rewards, next_q_values, dones, gamma):
    return rewards + gamma * next_q_values * (1 - dones)

  def td_update(self, local_qnetwork, target_qnetwork, interpolation_parameter):
    local_parameters = list(local_qnetwork.parameters())
    target_parameters = list(target_qnetwork.parameters())

    # Loop through each parameter index
    for i in range(len(local_parameters)):
        local_param = local_parameters[i]
        target_param = target_parameters[i]

        # Perform the soft update (temporal difference update)
        updated_param = interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data
        target_param.data.copy_(updated_param)

  def train(self, env, number_episodes, max_time_steps, epsilon_starting_value, epsilon_ending_value, epsilon_decay_value):
      episode_scores = []
      moving_avg_scores = []
      epsilon_values = []
      episode_lengths = []
      timestep_counts = []
      cumulative_timesteps = 0

      epsilon = epsilon_starting_value

      for i in range(number_episodes):  # episodes
          if i > 2000:
              print("Training aborted: episode limit exceeded.")
              return (
                  [np.nan], [np.nan], [np.nan], [np.nan], [np.nan]
              )

          state, info = env.reset()
          score = 0

          for j in range(max_time_steps):
              action = self.get_action(state, epsilon)
              next_state, reward, done, _, _ = env.step(action)
              self.step(state, action, reward, next_state, done)
              state = next_state
              score += reward
              if done:
                  break

          episode_lengths.append(j + 1)
          episode_scores.append(score)
          cumulative_timesteps += (j + 1)
          timestep_counts.append(cumulative_timesteps)

          if len(episode_scores) >= 100:
              avg_score = np.mean(episode_scores[-100:])
          else:
              avg_score = np.mean(episode_scores)

          moving_avg_scores.append(avg_score)

          epsilon = max(epsilon_ending_value, epsilon * epsilon_decay_value)
          epsilon_values.append(epsilon)

          if i % 100 == 0:
              print(f"Episode {i} | Average Score (last 100 episodes): {avg_score:.2f}")
          if avg_score >= 200.0:
              break

      return moving_avg_scores, episode_scores, epsilon_values, episode_lengths, timestep_counts


In [8]:
def evaluate_agent(agent, env, n_eval_episodes=10):
    evaluation_rewards = []
    evaluation_lengths = []

    for episode in range(n_eval_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0
        steps = 0

        while not done:
            # Use greedy policy: epsilon = 0
            action = agent.get_action(state, epsilon=0.0)
            next_state, reward, done, _, _ = env.step(action.item())
            state = next_state
            total_reward += reward
            steps += 1

        evaluation_rewards.append(total_reward)
        evaluation_lengths.append(steps)

    mean_reward = np.mean(evaluation_rewards)
    std_reward = np.std(evaluation_rewards)

    return evaluation_rewards, evaluation_lengths, mean_reward, std_reward

In [9]:
from itertools import product
import os
import numpy as np
import pandas as pd
import torch

# === Define hyperparameter ranges ===
epsilon_start_values = [0.8, 0.9, 1.0]
gamma_values = [0.95, 0.97, 0.99]
batch_sizes = [125, 150, 175]

# === Fixed parameters ===
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
interpolation_parameter = 1e-3
replay_buffer_size = 100000
learning_rate = 5e-4
n_eval_episodes = 10

# === Save directory ===
base_dir = "/Users/maartendoekhie/Desktop/lunar_lander/results/sensitivity"
os.makedirs(base_dir, exist_ok=True)

# === Load trained agent once ===
agent = Agent(state_size, action_size)
agent.optimizer = optim.Adam(agent.local_qnetwork.parameters(), lr=learning_rate)

# Load trained Q-network weights
local_q_path = "/Users/maartendoekhie/Desktop/lunar_lander/models/local_qnetwork_DQN.pt"
target_q_path = "/Users/maartendoekhie/Desktop/lunar_lander/models/local_qnetwork_DQN.pt"

agent.local_qnetwork.load_state_dict(torch.load(local_q_path, map_location="cpu"))
agent.target_qnetwork.load_state_dict(torch.load(target_q_path, map_location="cpu"))

agent.local_qnetwork.eval()
agent.target_qnetwork.eval()

# === Results summary ===
results = []

# === Loop over all parameter combinations ===
for epsilon_starting_value, gamma, minibatch in product(epsilon_start_values, gamma_values, batch_sizes):
    folder_name = f"epsilon_{epsilon_starting_value}__gamma_{gamma}__batch_{minibatch}"
    save_folder = os.path.join(base_dir, folder_name)
    rewards_file = os.path.join(save_folder, "eval_rewards.npy")
    lengths_file = os.path.join(save_folder, "eval_lengths.npy")

    # Skip if this configuration was already completed
    if os.path.exists(rewards_file) and os.path.exists(lengths_file):
        print(f"Skipping {folder_name} (already done)")
        # Optionally load existing results to append to summary
        try:
            eval_rewards = np.load(rewards_file)
            eval_lengths = np.load(lengths_file)
            results.append({
                'epsilon_start': epsilon_starting_value,
                'gamma': gamma,
                'batch_size': minibatch,
                'mean_reward': np.mean(eval_rewards),
                'std_reward': np.std(eval_rewards),
                'mean_length': np.mean(eval_lengths)
            })
        except Exception:
            print(f"⚠️ Could not reload saved results for {folder_name}")
        continue

    print(f"\n--- Evaluating for ε_start={epsilon_starting_value}, γ={gamma}, minibatch={minibatch} ---")
    os.makedirs(save_folder, exist_ok=True)

    # Update agent evaluation parameters
    agent.batch_size = minibatch
    agent.gamma = gamma

    # Evaluate agent
    try:
        eval_rewards, eval_lengths, mean_reward, std_reward = evaluate_agent(
            agent, env, n_eval_episodes=n_eval_episodes
        )
        mean_length = np.mean(eval_lengths)
    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        eval_rewards = np.full(n_eval_episodes, np.nan)
        eval_lengths = np.full(n_eval_episodes, np.nan)
        mean_reward = np.nan
        std_reward = np.nan
        mean_length = np.nan

    # Save numpy arrays
    np.save(rewards_file, eval_rewards)
    np.save(lengths_file, eval_lengths)

    # Append to results summary
    results.append({
        'epsilon_start': epsilon_starting_value,
        'gamma': gamma,
        'batch_size': minibatch,
        'mean_reward': mean_reward,
        'std_reward': std_reward,
        'mean_length': mean_length
    })

# === Save summary table ===
results_df = pd.DataFrame(results)
results_path = "/Users/maartendoekhie/Desktop/lunar_lander/results/sensitivity_results.csv"
results_df.to_csv(results_path, index=False)

# === Optional: rank by mean reward ===
sorted_df = results_df.sort_values(by='mean_reward', ascending=False)
print("\nTop configurations by mean reward:")
print(sorted_df.head())

Skipping epsilon_0.8__gamma_0.95__batch_125 (already done)
Skipping epsilon_0.8__gamma_0.95__batch_150 (already done)
Skipping epsilon_0.8__gamma_0.95__batch_175 (already done)
Skipping epsilon_0.8__gamma_0.97__batch_125 (already done)
Skipping epsilon_0.8__gamma_0.97__batch_150 (already done)
Skipping epsilon_0.8__gamma_0.97__batch_175 (already done)
Skipping epsilon_0.8__gamma_0.99__batch_125 (already done)
Skipping epsilon_0.8__gamma_0.99__batch_150 (already done)
Skipping epsilon_0.8__gamma_0.99__batch_175 (already done)
Skipping epsilon_0.9__gamma_0.95__batch_125 (already done)
Skipping epsilon_0.9__gamma_0.95__batch_150 (already done)
Skipping epsilon_0.9__gamma_0.95__batch_175 (already done)
Skipping epsilon_0.9__gamma_0.97__batch_125 (already done)
Skipping epsilon_0.9__gamma_0.97__batch_150 (already done)
Skipping epsilon_0.9__gamma_0.97__batch_175 (already done)
Skipping epsilon_0.9__gamma_0.99__batch_125 (already done)
Skipping epsilon_0.9__gamma_0.99__batch_150 (already don